In [1]:
import numpy as np
from PIL import Image, ImageEnhance
from plantcv import plantcv as pcv
from plantcv.parallel import WorkflowInputs
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import experimental, Conv2D, MaxPooling2D, Dense, Flatten
import os
import argparse

2024-02-06 14:20:53.834562: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-06 14:20:53.836621: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-06 14:20:53.857999: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-06 14:20:53.858020: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-06 14:20:53.858683: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
def loadDataset(path, img_size, batch_size):
    return tf.keras.preprocessing.image_dataset_from_directory(
        path,
        shuffle=True,
        image_size=(img_size, img_size)
    )

In [3]:
def removeBack(img, size_fill, enhance_val, buffer_size):
    img_img = Image.fromarray(img, mode="RGB")
    contr_img = ImageEnhance.Contrast(img_img).enhance(enhance_val)
    gray_img = pcv.rgb2gray_lab(rgb_img=np.array(contr_img), channel='a')
    thresh = pcv.threshold.triangle(
        gray_img=gray_img, object_type="dark", xstep=100)
    edge_ok = pcv.fill(bin_img=thresh, size=5000)
    mask = pcv.fill(bin_img=pcv.invert(gray_img=edge_ok), size=size_fill)
    contours, _ = cv2.findContours(
        mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    mask_buf = mask.copy()
    if (len(contours)):
        cv2.drawContours(mask_buf,
                         contours[np.argmax([len(c) for c in contours])],
                         -1, (0, 0, 0), buffer_size)
    if ([mask_buf[0, 0], mask_buf[0, -1],
         mask_buf[0, -1], mask_buf[-1, 0]] == [0, 0, 0, 0]):
        mask_buf[0:11, 0:11] = 255
        mask_buf[-11:, -11:] = 255
        mask_buf[0:11, -11:] = 255
        mask_buf[-11:, 0:11] = 255
    mask_buf[0:1, :] = 255
    mask_buf[-1:, :] = 255
    mask_buf[:, 0:1] = 255
    mask_buf[:, -1:] = 255
    mask_buf = pcv.fill(bin_img=mask_buf, size=size_fill)
    img_modified = np.ones_like(img) * 255
    img_modified[mask_buf == 0] = img[mask_buf == 0]
    return img_modified

In [4]:
def processImgDataSet(path):
    img_path_list = [
         [[foldername, fn, '/'.join(
              [e for e in foldername.split("/") if e not in ["..", "."]])]
          for fn in filenames]
         for foldername, subdirectory, filenames in os.walk(path)
         if len(filenames)]
    img_path_list = np.array([element for sous_liste in
                              img_path_list for element in sous_liste])
    img_array = np.array(
         [np.array(Image.open(str(img_path[0] + "/" + img_path[1]), "r"))
          for img_path in img_path_list])
    img_back_removed = [removeBack(img, 5000, 1, 10) for img in img_array]
    img_back_removed_IMG = [Image.fromarray(img_array)
                            for img_array in img_back_removed]
    [os.makedirs("increased/" + path[2], exist_ok=True)
     for path in img_path_list]
    [img.save("increased/" + path[2] + "/" + path[1])
     for path, img in zip(img_path_list, img_back_removed_IMG)]
    return

In [5]:
def get_dataset_partition_tf(ds, train_split=0.85,
                             shuffle=True, shuffle_size=10000):
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    len_train_dataset = int(len(ds) * train_split)
    train_dataset = ds.take(len_train_dataset)
    cv_dataset = ds.skip(len_train_dataset)
    return train_dataset, cv_dataset

In [6]:
batch_size = 32
epochs = 15
path = "increased"
save_dir = ""
save_name = "learnings"
img_size = 256
input_shape = (img_size, img_size, 3)

In [7]:
dataset = loadDataset("../increased/leaves/images", img_size, batch_size)
train_ds, validation_ds = get_dataset_partition_tf(dataset)
train_ds = train_ds.cache().shuffle(1000).prefetch(
    buffer_size=tf.data.AUTOTUNE)
validation_ds = validation_ds.cache().shuffle(1000).prefetch(
    buffer_size=tf.data.AUTOTUNE)
class_names = dataset.class_names

Found 12175 files belonging to 8 classes.


In [8]:
model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
            MaxPooling2D((2, 2)),
            Conv2D(32, (3, 3), activation='relu'),
            MaxPooling2D((2, 2)),
            Flatten(),
            Dense(64, activation='relu'),
            Dense(len(class_names), activation='softmax')
        ])

In [9]:
model.build(input_shape=input_shape)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(
        from_logits=False),
    optimizer='adam',
    metrics=['accuracy'])

In [11]:
model.fit(
    train_ds,
    epochs=epochs,
    batch_size = 32,
    verbose=1,
    validation_data=validation_ds
)

Epoch 1/15
323/323 [==============================] - ETA: 0s - loss: 1.3721 - accuracy: 0.5587